# Menjalankan skrip sebagai pekerjaan alur

Alur memungkinkan Anda mengelompokkan beberapa langkah ke dalam satu alur kerja. Anda dapat membangun alur dengan komponen. Setiap komponen mencerminkan skrip Python untuk dijalankan. Komponen didefinisikan dalam file YAML yang menentukan skrip dan cara menjalankannya. 

## Sebelum Anda memulai

Anda akan memerlukan versi terbaru paket  **azureml-ai-ml** untuk menjalankan kode di notebook ini. Jalankan sel di bawah ini untuk memverifikasi bahwa sel diinstal.

> **Catatan:**
> Jika paket **azure-ai-ml** tidak diinstal, jalankan `pip install azure-ai-ml` untuk menginstalnya.

In [ ]:
## Menyambungkan ke ruang kerja Anda

Dengan paket SDK yang diperlukan terinstal, sekarang Anda siap untuk terhubung ke ruang kerja Anda.

Untuk menyambungkan ke ruang kerja, kita memerlukan parameter pengidentifikasi - ID langganan, nama grup sumber daya, dan nama ruang kerja. Nama grup sumber daya dan nama ruang kerja sudah diisi untuk Anda. Anda hanya memerlukan ID langganan untuk menyelesaikan perintah.

Untuk menemukan parameter yang diperlukan, klik nama langganan dan ruang kerja di kanan atas Studio. Panel akan terbuka di sebelah kanan.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Salin ID langganan dan ganti **YOUR-SUBSCRIPTION-ID** dengan nilai yang Anda salin. </p>

## Membuat skrip

Anda akan membuat alur dengan dua langkah:

1. **Siapkan data**: Perbaiki data yang hilang dan normalisasi data.
1. **Melatih model**: Melatih model klasifikasi pohon keputusan.

Jalankan sel berikut untuk membuat folder **src** dan dua skrip.

In [ ]:
## Menentukan komponen

Untuk menentukan komponen, Anda perlu menentukan:

- **Metadata**: *nama*, *nama tampilan*, *versi*, *deskripsi*, *jenis* dll. Metadata membantu menjelaskan dan mengelola komponen.
- **Antarmuka**: *input* dan *output*. Misalnya, komponen pelatihan model akan mengambil data pelatihan dan tingkat regularisasi sebagai input, dan menghasilkan file model terlatih sebagai output. 
- **Perintah, kode & lingkungan**: *perintah*, *kode* , dan *lingkungan* untuk menjalankan komponen. Perintah adalah perintah shell untuk menjalankan komponen. Kode biasanya mengacu pada direktori kode sumber. Lingkungan bisa menjadi lingkungan AzureML (dibuat atau dibuat khusus), gambar docker, atau lingkungan conda.

Jalankan sel berikut untuk membuat YAML untuk setiap komponen yang ingin Anda jalankan sebagai langkah alur.

In [ ]:
## Memuat komponen

Setelah menentukan setiap komponen, Anda dapat memuat komponen dengan merujuk ke file YAML. 

## Membangun alur

Setelah membuat dan memuat komponen, Anda dapat membangun alur. Anda akan menyusun dua komponen ke dalam alur. Pertama, Anda akan ingin `prep_data` komponen berjalan. Output komponen pertama harus menjadi input komponen `train_decision_tree`kedua , yang akan melatih model.

Fungsi ini `diabetes_classification` mewakili alur lengkap. Fungsi mengharapkan satu variabel input: `pipeline_job_input`. Aset data dibuat selama penyiapan. Anda akan menggunakan aset data terdaftar sebagai input alur. 

In [ ]:
Anda dapat mengambil konfigurasi pekerjaan alur dengan mencetak `pipeline_job` objek :

In [ ]:
Anda dapat mengubah parameter konfigurasi pekerjaan alur dengan merujuk ke parameter dan menentukan nilai baru:

In [ ]:
## Mengirimkan pekerjaan alur

Terakhir, ketika Anda telah membangun alur dan mengonfigurasi pekerjaan alur untuk berjalan sesuai kebutuhan, Anda dapat mengirimkan pekerjaan alur:

## Define the components

To define the component you need to specify:

- **Metadata**: *name*, *display name*, *version*, *description*, *type* etc. The metadata helps to describe and manage the component.
- **Interface**: *inputs* and *outputs*. For example, a model training component will take training data and the regularization rate as input, and generate a trained model file as output. 
- **Command, code & environment**: the *command*, *code* and *environment* to run the component. Command is the shell command to execute the component. Code usually refers to a source code directory. Environment could be an AzureML environment (curated or custom created), docker image or conda environment.

Run the following cells to create a YAML for each component you want to run as a pipeline step.

In [ ]:
%%writefile prep-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_file
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python prep-data.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}

In [ ]:
%%writefile train-model.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train a decision tree classifier model
version: 1
type: command
inputs:
  training_data: 
    type: uri_file
  reg_rate:
    type: number
    default: 0.01
outputs:
  model_output:
    type: mlflow_model
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python train-model.py 
  --training_data ${{inputs.training_data}} 
  --reg_rate ${{inputs.reg_rate}} 
  --model_output ${{outputs.model_output}} 

## Load the components

Now that you have defined each component, you can load the components by referring to the YAML files. 

In [ ]:
from azure.ai.ml import load_component
parent_dir = ""

prep_data = load_component(source=parent_dir + "./prep-data.yml")
train_decision_tree = load_component(source=parent_dir + "./train-model.yml")

## Build the pipeline

After creating and loading the components, you can build the pipeline. You'll compose the two components into a pipeline. First, you'll want the `prep_data` component to run. The output of the first component should be the input of the second component `train_decision_tree`, which will train the model.

The `diabetes_classification` function represents the complete pipeline. The function expects one input variable: `pipeline_job_input`. A data asset was created during setup. You'll use the registered data asset as the pipeline input. 

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = prep_data(input_data=pipeline_job_input)
    train_model = train_decision_tree(training_data=clean_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-data:1"))

You can retrieve the configuration of the pipeline job by printing the `pipeline_job` object:

In [ ]:
print(pipeline_job)

You can change any parameter of the pipeline job configuration by referring to the parameter and specifying the new value:

In [ ]:
# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"
# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

## Submit the pipeline job

Finally, when you've built the pipeline and configured the pipeline job to run as required, you can submit the pipeline job:

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes"
)
pipeline_job